In [35]:
import pather
import time
import importlib
import pdfparser
import main
import reader
import preprocessing
import clustering
import searcher
import summarizer

In [37]:
search_engine = main.SearchEngine(r'C:\Users\sergiojesus\Desktop\Document_Clustering\CV', lang='pt')

In [39]:
%time results = search_engine.run_search(['C++'], num_returns=10, summary=True)

Results:


,File,Link,Score,Summary,Results
0,André Joaquim Barbosa de Oliveira_05_2017,CV\maio 2017\André Joaquim Barbosa de Oliveira_05_2017.pdf,1,De Março 2008 até Julho de 2012 ( 4 anos ) 5 anos Diretor de serviços Eletrotécnicos e Manutenção Eletromecânica De Março a Novembro de 2013 ( 1 ano ) Submissão de patente portuguesa nº 103807 intitulada “Sistema de recolha de bolas de golfe totalmente autónomo ou remotamente operado”.,"3 anos Programador C/C++ linux Programador C/C++ na SDILAB Investigação e desenvolvimento de software. Desenvolvimento adaptado aos serviços de venda e validação de titulos de transporte para rede metropolitana de LISBOA com uso de sistemas georeferenciados, programação de validadores AEP com leitura de cartões CALYPSO linguagem de programação C/C++."
1,Daniel José Silva Teixeira,CV\11-Novembro 2015\Daniel José Silva Teixeira.pdf,0.957629,"▪ Competências de programação (MatLab, MySql, C#, C++) Produção oral ▪ Alguns conhecimentos de projeto de redes eletrónicas. Foi uma experiência pessoal enriquecedora. ▪ Projeto de dispositivos médicos; Competências de organização Boa capacidade de organização do trabalho Experiência de eramus em uma excelente faculdade na Turquia. Foi uma boa experiência pessoal.","▪ Competências de programação (MatLab, MySql, C#, C++) Maio de 2015 a atualidade Programar uma aplicação de Re Rating na Vodafone em C/C++ ▪ Programação (C++, Fortran, Matlab); Desenvolvimento de Software em C/C++"
2,Rui Moreira,CV\10-outubro 2016\Rui Moreira.pdf,0.889691,Sector de atividade: Ensino / Formação técnico profissional nas áreas de profissional nas áreas de Responsável por angariação e orientação de formação em contexto de trabalho em empresas do ramo ▪ Implementação de projeto técnico certificado de telecomunicações em edifícios de acordo com a regulamentação técnica em vigor da ANACOM e recomendações municipais,"▪ Formação de programação em V. BASIC, C++, bases de dados mySQL e ▪ Programação em diversas plataformas (Omnis, Java em diversas plataformas (Omnis, Java J2SE, C++, C#, .Net, ▪ Programação de Computadores ( VBasic, C++, JAVA, SQL server, PHP, V. BASIC, C++, bases de dados mySQL e Tigerlogic Omnis, C++, C#, ASSEMBLY, HTML;"
3,Jorge Ferrão CV,CV\geral_2018\Jorge Ferrão CV.pdf,0.716885,"Tenho conhecimentos de programação em C/C++, Java, C#, Python e Javascript, e experiencia com computação gráfica, visão por computador, desenvolvimento de aplicações multiplataforma, programação de microcontroladores, desenvolvimento de jogos, desenvolvimento de sistemas de informação, aprendizagem automática e desenvolvimento web","Jogo de damas (2014) Jogo de damas humano vs computador, com realidade aumentada. Desenvolvimento de sistema de visão para deteção de tabuleiros de damas. OpenCV, C++ Software developer C++ / ROS para o robô autónomo WiiGo Algoritmos localização e navegação autónoma Ferramentas para visualização de dados Framework para criação de animação vetorial em Java. Java, LibGDX, JNI, C++ Tenho conhecimentos de programação em C/C++, Java, C#, Python e Javascript, e experiencia com computação gráfica, visão por computador, desenvolvimento de aplicações multiplataforma, programação de microcontroladores, desenvolvimento de jogos, desenvolvimento de sistemas de informação, aprendizagem automática e desenvolvimento web"
4,José Manuel Miranda Ferrão_12-2017,CV\Dezembro 2017\José Manuel Miranda Ferrão_12-2017.pdf,0.716885,"Tenho conhecimentos de programação em C/C++, Java, C#, Python e Javascript, e experiencia com computação gráfica, visão por computador, desenvolvimento de aplicações multiplataforma, programação de microcontroladores, desenvolvimento de jogos, desenvolvimento de sistemas de informação, aprendizagem automática e desenvolvimento web","Jogo de damas (2014) Jogo de damas humano vs computador, com realidade aumentada. Desenvolvimento de sistema de visão para deteção de tabuleiros de damas. OpenCV, C++ Software developer C++ / ROS para o robô autónomo WiiGo Algoritmos localização e na

Wall time: 259 ms
